In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

In [2]:
# Load and preprocess data
def preprocess_data():
    data = yf.download("EURUSD=x", period="5d", interval="1h")
    data.index = pd.to_datetime(data.index)
    data = data.drop(columns=['Adj Close', 'Volume'])
    data
    
    # Initialize Bollinger Bands Indicator
    indicator_bb = BollingerBands(close=data["Close"], window=20, window_dev=2)
    data['bb_bbm'] = indicator_bb.bollinger_mavg()
    data['hband'] = indicator_bb.bollinger_hband()
    data['lband'] = indicator_bb.bollinger_lband()
    
    # Initialize RSI Indicator
    indicator_rsi = RSIIndicator(close=data["Close"], window=14)
    data['rsi'] = indicator_rsi.rsi()
    
    return data

In [3]:
# Define the trading strategy
def trading_strategy(data, initial_cash=10000, max_intraday_loss=0.5):
    cash = initial_cash
    position = 0  # 0 means no position, 1 means long, -1 means short
    data['cash'] = cash
    data['position'] = position
    data['portfolio_value'] = cash
    highest_value = cash
    lowest_value = cash
    
    rsi_oversold = 30
    rsi_overbought = 70
    
    for i in range(1, len(data)):
        if position == 0:
            # Check for long entry
            if (data['rsi'].iloc[i] > rsi_oversold and data['rsi'].iloc[i-1] <= rsi_oversold) and (data['Close'].iloc[i] > data['bb_bbl'].iloc[i] and data['Close'].iloc[i-1] <= data['bb_bbl'].iloc[i-1]):
                position = 1
                entry_price = data['Close'].iloc[i]
                print(f"Entering long at {entry_price} on {data.index[i]}")
            # Check for short entry
            elif (data['rsi'].iloc[i] < rsi_overbought and data['rsi'].iloc[i-1] >= rsi_overbought) and (data['Close'].iloc[i] < data['bb_bbh'].iloc[i] and data['Close'].iloc[i-1] >= data['bb_bbh'].iloc[i-1]):
                position = -1
                entry_price = data['Close'].iloc[i]
                print(f"Entering short at {entry_price} on {data.index[i]}")
        elif position == 1:
            # Check for long exit
            if (data['rsi'].iloc[i] < rsi_overbought and data['rsi'].iloc[i-1] >= rsi_overbought) or (data['Close'].iloc[i] / entry_price - 1) < -max_intraday_loss:
                position = 0
                cash += (data['Close'].iloc[i] - entry_price) * 10  # Assuming we buy/sell 10 units
                highest_value = max(highest_value, cash)
                lowest_value = min(lowest_value, cash)
                print(f"Exiting long at {data['Close'].iloc[i]} on {data.index[i]}")
        elif position == -1:
            # Check for short exit
            if (data['rsi'].iloc[i] > rsi_oversold and data['rsi'].iloc[i-1] <= rsi_oversold) or (entry_price / data['Close'].iloc[i] - 1) < -max_intraday_loss:
                position = 0
                cash += (entry_price - data['Close'].iloc[i]) * 10  # Assuming we buy/sell 10 units
                highest_value = max(highest_value, cash)
                lowest_value = min(lowest_value, cash)
                print(f"Exiting short at {data['Close'].iloc[i]} on {data.index[i]}")
        
        data.at[data.index[i], 'cash'] = cash
        data.at[data.index[i], 'position'] = position
        data.at[data.index[i], 'portfolio_value'] = cash + position * (data['Close'].iloc[i] - entry_price) * 10
    
    return data

In [4]:
# Run the backtest
if __name__ == "__main__":
    data = preprocess_data()
    data
    results = trading_strategy(data)
    
    # Plot the results
    plt.figure(figsize=(12, 8))
    plt.plot(results['portfolio_value'], label='Portfolio Value')
    plt.title('Portfolio Value Over Time')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plt.show()

[*********************100%%**********************]  1 of 1 completed


UnboundLocalError: cannot access local variable 'entry_price' where it is not associated with a value